In [ ]:
import sys
print(sys.path)
import os
import requests
import logging
from dotenv import load_dotenv
from pymongo import MongoClient
from openai import OpenAI
import datetime
import pandas as pd

# Load environment variables
load_dotenv()

# Retrieve environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
rawg_api_key = os.getenv("RAWG_API_KEY")
twitch_client_id = os.getenv("TWITCH_CLIENT_ID")
twitch_client_secret = os.getenv("TWITCH_CLIENT_SECRET")
twitch_token_url = os.getenv("TWITCH_TOKEN_URL")
mongo_uri = os.getenv("MONGODB_URI")

client = OpenAI(api_key=openai_api_key)

Ensure Variables are Loaded Correctly

In [ ]:
if not all([openai_api_key, rawg_api_key, twitch_client_id, twitch_client_secret, twitch_token_url, mongo_uri]):
    raise ValueError("One or more environment variables are missing or not set correctly")

Configure Logging

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

Connect to MongoDB

In [ ]:
mongo_client = MongoClient(mongo_uri)
db = mongo_client["Wingman"]
user_id_collection = db["userID"]
questions_collection = db["question"]

Load the CSV Files

In [ ]:
video_games_df = pd.read_csv('data/Video Games Data.csv')
data_dictionary_df = pd.read_csv('data/vg_data_dictionary.csv')

Search Games by Genre

In [ ]:
def search_games_by_genre(genre):
    results = video_games_df[video_games_df['Genre'].str.contains(genre, case=False, na=False)]
    return results

Search Games by Name

In [ ]:
def search_game_by_name(game_name):
    results = video_games_df[video_games_df['title'].str.contains(game_name, case=False)]
    return results

Get Access Token

In [ ]:
import time
import datetime

token_info = {"access_token": None, "expires_at": None}

def get_access_token():
    """ Fetch access token using Twitch credentials. """
    data = {
        'client_id': twitch_client_id,
        'client_secret': twitch_client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.post(twitch_token_url, data=data)
    response.raise_for_status()
    token_data = response.json()
    expires_in = token_data['expires_in']
    token_info['access_token'] = token_data['access_token']
    token_info['expires_at'] = datetime.datetime.now() + datetime.timedelta(seconds=expires_in)
    logging.info("New access token fetched and stored.")

def check_token():
    """ Check if the token is expired and refresh it if needed. """
    if token_info['expires_at'] is None or token_info['expires_at'] <= datetime.datetime.now():
        logging.info("Token has expired, fetching a new one...")
        get_access_token()
    else:
        logging.info("Token is still valid.")

# Example usage
try:
    # Always check the token before making a request
    check_token()
    access_token = token_info['access_token']
    if access_token:
        print(f"Using access token: {access_token}")
        # Proceed with using the access token for API requests
    else:
        print("Failed to obtain access token.")
except Exception as e:
    print(f"Error obtaining or using access token: {e}")

print("Token URL:", twitch_token_url)

Make a Request to IGDB API

In [ ]:
def fetch_igdb_data(access_token, client_id):
    headers = {
        'Client-ID': client_id,
        'Authorization': f'Bearer {access_token}',
    }
    response = requests.post(
        'https://api.igdb.com/v4/games',
        headers=headers,
        data='fields name,genres.name,platforms.name,release_dates.human; limit 10;'
    )
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}, {response.text}")

Fetch Game data from RAWG API

In [ ]:
def fetch_games_from_rawg(search_query):
    # Fetch the RAWG API key directly from environment variables within the function
    rawg_api_key = os.getenv("RAWG_API_KEY")
    # Construct the URL with the API key and the search query properly included
    url = f"https://api.rawg.io/api/games?key={rawg_api_key}&search={search_query}"
    # Use a GET request to fetch data
    response = requests.get(url)
    # Check the response status and handle accordingly
    if response.status_code == 200:
        return response.json()['results']  # Make sure to return the results part of the JSON
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}, {response.text}")

Fetch Data from both RAWG, IGDB and CSV Files

In [ ]:
def fetch_data_from_all_sources(game_name):
    try:
        check_token()
        access_token = token_info['access_token']
        
        rawg_data = fetch_games_from_rawg(game_name)
        igdb_data = fetch_igdb_data(access_token, twitch_client_id)
        
        # Assuming search_game_by_name is commented out for now, as per your request
        # csv_data = search_game_by_name(game_name)
        # csv_games = [f"{row['title']} (Released: {row['release_date']}, Platforms: {row['console']}, Genres: {row['genre']})" for index, row in csv_data.iterrows()]

        rawg_games = [
            f"{game['name']} (Released: {game['released']}, Platforms: {game['platforms']}, Genres: {game['genres']})"
            for game in rawg_data
        ]
        
        igdb_games = [
            f"{game['name']} (Released: {game['release_dates'][0]['date'] if game.get('release_dates') else 'Unknown'}, Platforms: {', '.join([platform['name'] for platform in game.get('platforms', [])])}, Genres: {', '.join([genre['name'] for genre in game.get('genres', [])])})"
            for game in igdb_data
        ]
        
        # Combine responses into a single output
        combined_response = ""
        
        if rawg_games:
            combined_response += f"Based on your interest in {game_name}, here are some games you might enjoy from RAWG:\n" + "\n".join(rawg_games[:10]) + "\n"
        
        if igdb_games:
            combined_response += f"\nFrom IGDB, you might also like:\n" + "\n".join(igdb_games[:10]) + "\n"
        
        # If CSV data were to be used
        # if csv_games:
        #     combined_response += f"\nFrom our CSV dataset, you might also like:\n" + "\n".join(csv_games[:10]) + "\n"
        
        # Handle case when no relevant data is found
        if not combined_response.strip():
            combined_response = "No relevant game information found in any database."
        
        return combined_response
    
    except Exception as e:
        logging.error(f"Error fetching data from APIs: {e}")
        return "Failed to fetch data due to an error."

Check if API Key is loaded

In [ ]:
if openai_api_key is None:
    raise ValueError("API key not found. Please set the OPENAI_API_KEY environment variable.")

AI Game Analytics 

In [ ]:
def setup_openai_assistant():
    try:
        assistant = client.beta.assistants.create(
            name="Video Game Analytics Assistant",
            instructions="""
            You are an AI assistant specializing in video games. You can provide detailed analytics and insights into gameplay, helping players track their progress and identify areas for improvement. 
            You can answer questions about game completion times, strategies to progress past difficult sections, fastest speedrun times, and general tips and tricks.
            For example:
            - "How long would it take to fully complete Super Mario Bros. on NES?"
            - "How do I progress past the Water Temple in The Legend of Zelda: Ocarina of Time?"
            - "What is the fastest speedrun time for Sonic the Hedgehog?"
            - "Give me tips and tricks to improve my gameplay in Fortnite."
            - "Give me a guide on how to clear Chapter 3 in Paper Mario: The Thousand Year Door."
            - "What is the best strategy for completing Shrines in The Legend of Zelda: Breath of the Wild?"
            - "What games in terms of genre are similar to Super Metroid?"
            - "What games could you recommend to someone who is playing a Role-Playing Game for the first time?"
            - "What types of games are you familiar with?"
            -" What are some of the biggest challenges impacting the Global Video Game Industry?
            Use up-to-date information and provide the best possible answers to enhance the user's gaming experience.
            """,
            tools=[{"type": "code_interpreter"}],  # Add more tools if needed
            model="gpt-4o"
        )
        return assistant
    except Exception as e:
        logging.error(f"Failed to create assistant: {e}")
        return None

Create a Thred

In [ ]:
from openai import APIError

def create_thread():
    try:
        thread = client.beta.threads.create()
        logging.info(f"Thread created: {thread}")
        return thread
    except APIError as e:
        logging.error(f"Error creating thread: {e}")
        return None
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return None

Add a Message to a Thread

In [ ]:
def add_message_to_thread(thread_id, message_content):
    try:
        message = client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=message_content
        )
        logging.info(f"Message added to thread: {message}")
        return message
    except Exception as e:
        logging.error(f"Error adding message to thread: {e}")
        return None

Run the Assistant

In [ ]:
def run_assistant(thread_id, assistant_id):
    try:
        run = client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id
        )
        logging.info(f"Run created: {run}")
        return run
    except Exception as e:
        logging.error(f"Error running assistant: {e}")
        return None

Retreive and Display the Assistant's Response

In [ ]:
import time

def display_assistant_response(thread_id, run_id):
    try:
        time.sleep(15)  # Increase delay to allow more processing time
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        for message in reversed(messages.data):
            if message.role == "assistant":
                logging.info(f"Full Assistant Message: {message}")  # Log the full message for debugging
                if message.content and message.content[0].text and message.content[0].text.value:
                    response = message.content[0].text.value
                    logging.info(f"Assistant: {response}")
                    return response
                else:
                    logging.info("Assistant message does not have the expected content format.")
        logging.info("No response from assistant.")
    except Exception as e:
        logging.error(f"Error retrieving messages: {e}")

Get or Create User in MongoDB

In [ ]:
def get_or_create_user(user_id):
    user = user_id_collection.find_one({"userId": user_id})
    if not user:
        user = {
            "userId": user_id,
            "conversations": []
        }
        user_id_collection.insert_one(user)
    return user

Save Interaction in MongoDB

In [ ]:
def save_interaction(user_id, question, response):
    interaction = {
        "userId": user_id,
        "question": question,
        "response": response,
        "timestamp": datetime.datetime.now()
    }
    questions_collection.insert_one(interaction)
    user_id_collection.update_one(
        {"userId": user_id},
        {"$push": {"conversations": interaction}},
        upsert=True
    )

Retrieve and Analyze Previous Interactions

In [ ]:
import re

def get_previous_interactions(user_id):
    user = user_id_collection.find_one({"userId": user_id})
    if user and "conversations" in user:
        return user["conversations"]
    return []

# Generate Recommendations Based on Previous Interactions
def generate_recommendations(previous_interactions):
    recommendations = []
    keywords = {
        "role-playing game": "Try playing 'Final Fantasy VII'",
        "rpg": "Try playing 'Paper Mario: The Thousand-Year Door'",
        "action-adventure": "Try playing 'The Elder Scrolls V: Skyrim'",
        "hack and slash": "You might enjoy 'Devil May Cry 3'",
        "action rpg": "Try playing 'Xenoblade Chronicles'",
        "battle royale": "Try playing 'Fortnite'",
        "platformer": "Try playing 'Super Mario 64'",
        "survival horror": "Try playing 'Resident Evil 4'",
        "third-person shooter": "Try playing 'Splatoon 3'",
        "metroidvania": "Try playing 'Castlevania: Symphony of the Night'",
        "first-person shooter": "Try playing 'Bioshock Infinite'",
        "sandbox": "Try playing 'Minecraft'",
        "roguelike": "Try playing 'Hades'",
        "social simulation": "Try playing 'Animal Crossing: New Horizons'",
        "mmo": "Try playing 'Runescape'",
        "massively multiplayer online role-playing game": "Try playing 'World of Warcraft'",
        "moba": "Try playing 'League of Legends'",
        "multiplayer online battle arena": "Try playing 'Dota 2'",
        "puzzle-platformer": "Try playing 'Portal'",
        "fighting game": "Try playing 'Super Smash Bros. Ultimate'",
        "tactical role-playing game": "Try playing 'Fire Emblem: Awakening'",
        "tower defense": "Try playing 'Bloons TD 6'",
        "racing": "Try playing 'Forza Horizon 5'",
        "kart racing": "Try playing 'Mario Kart 8 Deluxe'",
        "rail shooter": "Try playing 'Star Fox 64'",
        "stealth": "Try playing 'Metal Gear Solid'",
        "run and gun": "Try playing 'Cuphead'",
        "turn-based strategy": "Try playing 'Advance Wars'",
        "4x": "Try playing 'Sid Meier's Civilization VI'",
        "sports": "Try playing 'Fifa 18'",
        "party": "Try playing 'Mario Party Superstars'",
        "rhythm": "Try playing 'Rock Band'",
        "point and click": "Try playing 'Five Night's at Freddy's'",
        "visual novel": "Try playing 'Phoenix Wright: Ace Attorney'",
        "real-time strategy": "Try playing 'Command & Conquer'",
        "beat 'em up": "Try playing 'Streets of Rage 4'",
        "puzzle": "Try playing 'Tetris'",
        "turn-based tactics": "Try playing 'XCOM: Enemy Unknown'",
        "interactive story": "Try playing 'The Stanley Parable'",
        "maze": "Try playing 'Pac-Man'",
        "game creation system": "Try playing 'Roblox'",
        "level editor": "Try playing 'Super Mario Maker'",
        "endless runner": "Try playing 'Temple Run'",
        "digital collectible card game": "Try playing 'Yu-Gi-Oh! Master Duel'",
        "exergaming": "Try playing 'Wii Fit'",
        "immersive sim": "Try playing 'Deathloop'",
        "tile-matching": "Try playing 'Bejeweled'",
        "text based": "Try playing 'The Oregon Trail'",
        "augmented reality": "Try playing 'Pokémon Go'",
        "action-adventure": "Try playing 'The Last of Us'"
    }

    for interaction in previous_interactions:
        for keyword, recommendation in keywords.items():
            if re.search(keyword, interaction["question"], re.IGNORECASE):
                if recommendation not in recommendations:
                    recommendations.append(recommendation)
    
    return recommendations

Generate Response:

In [ ]:
def main():
    try:
        assistant = setup_openai_assistant()
        if assistant is None:
            raise Exception("Assistant could not be created.")

        user_id = input("Enter your user ID: ")
        question = input("Enter your gameplay data or question for analysis: ")

        # Get or create user in MongoDB
        user = get_or_create_user(user_id)
        
        thread = create_thread()
        if thread:
            if "similar to" in question.lower():
                game_name = question.split("similar to ")[1].strip()
                response = fetch_data_from_all_sources(game_name)
                print(response)
                save_interaction(user_id, question, response)
            elif "genre" in question.lower():
                genre = question.split("genre ")[1].strip()
                response = fetch_data_from_all_sources(genre)  # Use the generalized fetch function
                print(response)
                save_interaction(user_id, question, response)
            else:
                # For all other types of questions
                message = add_message_to_thread(thread.id, question)
                if message:
                    run = run_assistant(thread.id, assistant.id)
                    if run:
                        response = display_assistant_response(thread.id, run.id)
                        if response:
                            save_interaction(user_id, question, response)
                        else:
                            # If the assistant fails to provide a response, fetch from other sources
                            response = fetch_data_from_all_sources(question)
                            print(response)
                            save_interaction(user_id, question, response)
                    else:
                        logging.error("Failed to run assistant")
                else:
                    logging.error("Failed to add message to thread")
        else:
            logging.error("Failed to create thread")

        # Retrieve and analyze previous interactions
        previous_interactions = get_previous_interactions(user_id)
        recommendations = generate_recommendations(previous_interactions)
        if recommendations:
            recommendations_str = ", ".join(recommendations)
            print(f"Recommendations based on your previous interactions: {recommendations_str}")

    except Exception as e:
        logging.error(f"An error occurred in the main function: {e}")

if __name__ == "__main__":
    main()